In [3]:
import pandas as pd
import numpy as np

#local 読み込み
dataPath = './datasets/'
titlePath = './data/titledata/'
keyPath = './data/keyworddata/'
storyPath = './data/storydata/'
allPath =  './data/all/'

test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

In [15]:
name = "sudachi"

title_mecab_data = pd.read_csv(titlePath+'sudachi.csv')
key_mecab_data = pd.read_csv(keyPath+'sudachi.csv' )
story_mecab_data = pd.read_csv(storyPath+'sudachi.csv')

data = pd.DataFrame()
data["sentence"] = title_mecab_data["sudachi"] + " " + key_mecab_data["sudachi"] + " "+ story_mecab_data["sudachi"]


In [16]:
from gensim.models import word2vec

#[[文章の単語], [文章の単語]]を作成
title_dic = []
def make_title_dic(text):
  if text is np.nan:
    return
  title_dic.append(text.split(' '))
data['sentence'].map(make_title_dic)

#学習
model = word2vec.Word2Vec(title_dic,vector_size=50, min_count=5, window=5, epochs=20)

#作成した辞書をcsvに保存して可視化(処理には関係ない)
from gensim import corpora#辞書を作るためのもの
dictionary = corpora.Dictionary(title_dic)#textsをもとに辞書を作成します
dictionary.filter_extremes(no_below=5)#出現文書数が5回以下のものはさようなら
dictionary.save_as_text(allPath + 'dictionary.csv')#辞書を保存
dictionary.save


<bound method SaveLoad.save of <gensim.corpora.dictionary.Dictionary object at 0x000001635A979040>>

## ベクトルの平均

In [12]:
dimension = 100 #次元

#文章の分散表現を求める(単語ベクトルの平均)
def getSentenceVector(text):
  if text is np.nan:
    return np.array([np.nan for _ in range(dimension)])
  L = []
  for w in text.split(' '):
    if w in model.wv.key_to_index:
      L.append(model.wv.get_vector(key=w))
  if len(L)==0:
    return np.array([np.nan for _ in range(dimension)])
  return np.array(L).mean(axis=0)

#分散表現をnp.arrayの形に変換
def makeVectorArrayList(data, rowName):
  vec_list = []
  for index, row in data.iterrows():
    vec_list.append(getSentenceVector(row[rowName]))
  return np.array(vec_list)

data = pd.read_csv(titlePath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'title_vec_originalLearned_sudachi_'+ str(dimension) +'.csv')

print("title 完了")

data = pd.read_csv(storyPath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'story_vec_originalLearned_sudachi_'+ str(dimension) +'.csv')

print("story 完了")

data = pd.read_csv(keyPath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'key_vec_originalLearned_sudachi_'+ str(dimension) +'.csv')

title 完了
story 完了


## TF-IDF

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

data["sentence"] = title_mecab_data["sudachi"] + " " + key_mecab_data["sudachi"] + " "+ story_mecab_data["sudachi"]

dimension = 50 #次元

#単語辞書作成作成
docs = []
def make_docs(text):
  if text is np.nan:
    docs.append('')
  else:
    docs.append(text)
data['sentence'].map(make_docs)
# tf-idfの計算
vectorizer = TfidfVectorizer(max_df=0.9) #文書全体の90%以上で出現する単語は無視する
td_idf_list = vectorizer.fit_transform(docs)
td_idf_vocabulary =  vectorizer.vocabulary_

#文章の分散表現を求める(単語ベクトルの平均)
def getSentenceVector(text, id):
  if text is np.nan:
    return np.array([np.nan for _ in range(dimension)])
  L = []
  for w in text.split(' '):
    if w in model.wv.key_to_index:# modelにある時
      vector = model.wv.get_vector(key=w)
      if w in td_idf_vocabulary: # TD-IDFにある時
        td_idf = td_idf_list[id, td_idf_vocabulary[w]] #td-idf を取得
        L.append(list(map(lambda x: x*td_idf, vector))) # TD-IDFを掛ける
  if len(L)==0:
    return np.array([np.nan for _ in range(dimension)])
  return np.array(L).mean(axis=0)

#分散表現をnp.arrayの形に変換
def makeVectorArrayList(data, rowName):
  vec_list = []
  for id, row in data.iterrows():
    vec_list.append(getSentenceVector(row[rowName], id))
  return np.array(vec_list)

data = pd.read_csv(titlePath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'title_vec_tdidf_'+ str(dimension) +'.csv')

print("title 完了")

data = pd.read_csv(storyPath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'story_vec_tdidf_'+ str(dimension) +'.csv')

print("story 完了")

data = pd.read_csv(keyPath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'key_vec_tdidf_'+ str(dimension) +'.csv')




48522


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

docs = [
    'ドキュメント 集合 において ドキュメント の 単語 に 付けられる',
    '情報検索 において 単語 へ の 重み付け に 使える',
    'ドキュメント で 出現した すべて の 単語 の 総数',
]
vectorizer = TfidfVectorizer(max_df=0.9) # tf-idfの計算
X = vectorizer.fit_transform(docs)
w = vectorizer.vocabulary_
print(w['ドキュメント'], w['集合'], w['付けられる'], w['において'])
print(vectorizer.get_feature_names())
print(X[0])

2 9 3 1
['すべて', 'において', 'ドキュメント', '付けられる', '使える', '出現した', '情報検索', '総数', '重み付け', '集合']
  (0, 3)	0.4521233082633858
  (0, 1)	0.34385142961748544
  (0, 9)	0.4521233082633858
  (0, 2)	0.6877028592349709


C:\Users\ngnls\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
data["sentence"] = title_mecab_data["sudachi"] + " " + key_mecab_data["sudachi"] + " "+ story_mecab_data["sudachi"]
print(data["sentence"][0:1])

print(td_idf_vocabulary['暗い'])
print(td_idf_vocabulary['窓辺'])
# print(td_idf_list[44526])
# print(td_idf_list[44527])
print(td_idf_list[0, 44527])
print(type(td_idf_list))

td_idf = td_idf_list[44527][td_idf_vocabulary['保育園']] #td-idf を取得


0    暗い 窓辺 皇帝 北 宮 ファンタジー SF 天使 小人 猫 新しい 手直し 増し 宇宙 開...
Name: sentence, dtype: object
44527
53572
0.12312410733979824
<class 'scipy.sparse.csr.csr_matrix'>


IndexError: row index (28738) out of range